In [1]:
import numpy as np

def minMax_normalization(X):
    X = (X - X.min() ) / (X.max() - X.min())
    return X*100

<font size=5> load metadata, shape and dCam explanations </font>

In [2]:
ns = ["PseudoPeriodic","GaussianProcess", "AutoRegressive"]

In [3]:
# load dataset metadata
dir_name = "../data/synth_data/data/"
names=["SimulatedTestingMetaData_RareTime_PseudoPeriodic_F_20_TS_100_Positional_False.npy"
      ,"SimulatedTestingMetaData_RareTime_GaussianProcess_F_20_TS_100_Positional_False.npy",
      "SimulatedTestingMetaData_RareTime_AutoRegressive_F_20_TS_100_Positional_False.npy"]
metadata = [np.load(dir_name+n , allow_pickle=True).item() for n in names]

In [9]:
#load synthetic channel by channel
import copy
dir_name="rocket_shap_results/"
names=["PseudoPeriodic_Positional_False_Concatenated_200.npy","GaussianProcess_Positional_False_Concatenated_200.npy",
           "AutoRegressive_Positional_False_Concatenated_200.npy"]
exps_concat = [np.load(dir_name+n,allow_pickle=True).item() for n in names]
for i in range(3):
    exps_concat[i]["exps"] = exps_concat[i]["exps"].reshape(100,2,20,10).transpose([0,2,1,3])


In [10]:
#load synthetic channel by channel
dir_name="rocket_shap_results/"
names=["PseudoPeriodic_Positional_False_ChannelByChannel.npy","GaussianProcess_Positional_False_channelByChannel.npy",
           "AutoRegressive_Positional_False_channelByChannel.npy"]
exps = [np.load(dir_name+n,allow_pickle=True).item() for n in names]
exps[2].keys()
print( exps[2]["exps"].shape)

(100, 20, 2, 10)


In [11]:
exps[0]["exps"].shape

(100, 20, 2, 10)

In [12]:
# load dCAM synthetic
dir_name = "../explanations/dCAM_results/"
ns=["PseudoPeriodic_Positional_False_explenations.npy","GaussianProcess_Positional_False_explenations.npy"
    ,"AutoRegressive_Positional_False_explenations.npy"]#,"MP_old_explenations.npy"]
dCAM_data = [np.load(dir_name+n , allow_pickle=True) for n in ns]
dCAM_data[0][0].keys()

dict_keys(['dcam_tl', 'permutation_success_tl', 'dcam_ol', 'permutation_success_ol', 'ground_truth_label', 'output_label'])

In [13]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix,average_precision_score, roc_auc_score

<font size=5> evaluate starting from shap </font>

In [14]:
def average_ts(ts):
    avg_ts = np.zeros(shape=(20,10), dtype=ts.dtype)
    for i in range(avg_ts.shape[0]):
        for j in range(avg_ts.shape[1]):
            avg_ts[i][j] = np.average(ts[i][j*10 : (j+1)*10])
    return avg_ts

threshold = 50

for i in range(3):
    for method in ["SHAP_ch", "SHAP_co","dCAM"]:
        
        # initalise counters 
        metrics = {
            "macro": { "pr" : 0,"rec" :0, "f1":0, "PRAUC":0,"ROC":0},
            "micro" : {"tp":0,"fp":0,"fn":0}
        }
        
        #read the correct data structure 
        if method=="SHAP_ch":#exps_concat1
            # shap
            Xps = exps[i]["exps"]
            test_length = Xps.shape[0]
        elif method=="SHAP_co":
            # shap
            Xps = exps_concat[i]["exps"]
            test_length = Xps.shape[0]
        elif method=="dCAM":
            Xps = dCAM_data[i]

        # loop over test data
        test_length = Xps.shape[0]
        for j in range(test_length):
            if method=="SHAP_ch":
                # shap
                true_classes = exps[i]["ground_truth_labels"][i,:]
                out_classes = exps[i]["model_outputs"][i]
                orig_current_expl = Xps[i,:, int(out_classes[j]) , : ]
            elif method=="SHAP_co":
                # shap
                true_classes = exps_concat[i]["ground_truth_labels"]
                out_classes = exps_concat[i]["model_outputs"]
                orig_current_expl = Xps[i,:, int(out_classes[j]) , : ]
            elif method=="dCAM":
                if np.any(Xps[j]["dcam_ol"]==-1):
                    continue
                else:
                    true_classes = Xps[j]["ground_truth_label"]
                    out_classes =  Xps[j]["output_label"]
                    current_expl = Xps[j]["dcam_ol"]
                    orig_current_expl = average_ts(current_expl)
                    
            current_expl = minMax_normalization(orig_current_expl)
            importants = (current_expl>=threshold).astype(int)
            mask = np.zeros(shape=(20,10),dtype=int)
            mask[:10,1]=1
            importants = importants.flatten(); mask = mask.flatten()

            precision, recall, f1, support = precision_recall_fscore_support(
                mask, importants,beta=1.0, average="binary",pos_label=1)
            tn,fp,fn,tp= confusion_matrix(mask, importants).ravel()
            
            macro_PR_AUC = average_precision_score(mask,orig_current_expl.flatten(),average="macro")
            micro_PR_AUC = average_precision_score(mask,orig_current_expl.flatten(),average="weighted")

            macro_ROC_AUC = roc_auc_score(mask,orig_current_expl.flatten(),average="macro")
            micro_ROC_AUC = roc_auc_score(mask, orig_current_expl.flatten() ,average="weighted")
            
            metrics["macro"]["pr"]+=precision;
            metrics["macro"]["rec"]+=recall; 
            metrics["macro"]["f1"]+=f1; 
            metrics["micro"]["tp"]+=tp;
            metrics["micro"]["fp"]+=fp; 
            metrics["micro"]["fn"]+=fn; 
            
        #average
        micro_precision = metrics["micro"]["tp"] / (metrics["micro"]["tp"] +metrics["micro"]["fp"])
        micro_recall = metrics["micro"]["tp"] / (metrics["micro"]["tp"] +metrics["micro"]["fn"])
        micro_f1 = 2*(micro_precision*micro_recall) / (micro_precision + micro_recall)
        
        macro_precision = metrics["macro"]["pr"]/test_length 
        macro_recall =  metrics["macro"]["rec"]/test_length 
        # TODO how to compute macro f1?
        macro_f1 = 2*(macro_precision*macro_recall) / (macro_precision + macro_recall)       
        
        # TODO check macro f1
        print(ns[i] ,"\t",method, '\n\t macro: precision is {:.4f};\t  recall is {:.4f}; \t f1 is {:.4f}'.format(
            macro_precision,macro_recall,macro_f1),
            '\n\t micro: precision is {:.4f};\t  recall is {:.4f}; \t f1 is {:.4f}'.format(micro_precision,micro_recall,
            micro_f1),
              '\n\t macro AUC is {:.4f};\t micro AUC is {:.4f}'.format(macro_PR_AUC,micro_PR_AUC),
              '\n\t macro ROC is {:.4f};\t  micro ROC {:.4f};'.format(macro_ROC_AUC,micro_ROC_AUC),
              "\n")
    print("\n\n\n\n")

PseudoPeriodic_Positional_False_explenations.npy 	 SHAP_ch 
	 macro: precision is 0.1471;	  recall is 0.5000; 	 f1 is 0.2273 
	 micro: precision is 0.0500;	  recall is 0.5000; 	 f1 is 0.0909 
	 macro AUC is 0.9909;	 micro AUC is 0.9909 
	 macro ROC is 0.9995;	  micro ROC 0.9995; 

PseudoPeriodic_Positional_False_explenations.npy 	 SHAP_co 
	 macro: precision is 0.2083;	  recall is 0.5000; 	 f1 is 0.2941 
	 micro: precision is 0.0500;	  recall is 0.5000; 	 f1 is 0.0909 
	 macro AUC is 0.9497;	 micro AUC is 0.9497 
	 macro ROC is 0.9974;	  micro ROC 0.9974; 

PseudoPeriodic_Positional_False_explenations.npy 	 dCAM 
	 macro: precision is 0.5003;	  recall is 0.5000; 	 f1 is 0.5001 
	 micro: precision is 0.4995;	  recall is 0.5051; 	 f1 is 0.5023 
	 macro AUC is 0.6326;	 micro AUC is 0.6326 
	 macro ROC is 0.9732;	  micro ROC 0.9732; 






GaussianProcess_Positional_False_explenations.npy 	 SHAP_ch 
	 macro: precision is 0.3044;	  recall is 0.4460; 	 f1 is 0.3619 
	 micro: precision is 0.0

<font size=5> evaluate starting from dCAM </font>

In [15]:
def repeat_ts(ts):
    return np.repeat(ts,10).reshape(ts.shape[0],-1)

In [16]:
threshold = 50

for i in range(3):
    for method in ["SHAP_ch","SHAP_co","dCAM"]:
        
        # initalise counters 
        metrics = {
            "macro": { "pr" : 0,"rec" :0, "f1":0, "PRAUC":0,"ROC":0},
            "micro" : {"tp":0,"fp":0,"fn":0}
        }
        
        #read the correct data structure 
        if method=="SHAP_ch":
            # shap
            Xps = exps[i]["exps"]
            test_length = Xps.shape[0]
        elif method=="SHAP_co":
            # shap
            Xps = exps_concat[i]["exps"]
            test_length = Xps.shape[0]
        elif method=="dCAM":
            Xps = dCAM_data[i]

        # loop over test data
        test_length = Xps.shape[0]
        for j in range(test_length):
            if method=="SHAP_ch":
                # shap
                true_classes = exps[i]["ground_truth_labels"][0,:]
                out_classes = exps[i]["model_outputs"][0]
                orig_current_expl = repeat_ts( Xps[i,:, int(out_classes[j]) , : ] )
            elif method=="SHAP_co":
                # shap
                true_classes = exps_concat[i]["ground_truth_labels"]
                out_classes = exps_concat[i]["model_outputs"]
                orig_current_expl =  repeat_ts( Xps[i,:, int(out_classes[j]) , : ] )
            elif method=="dCAM":
                if np.any(Xps[j]["dcam_ol"]==-1):
                    continue
                else:
                    true_classes = Xps[j]["ground_truth_label"]
                    out_classes =  Xps[j]["output_label"]
                    orig_current_expl = Xps[j]["dcam_ol"]

                    
            current_expl = minMax_normalization(orig_current_expl)
            importants = (current_expl>=threshold).astype(int)
            mask = np.zeros(shape=(20,100),dtype=int)
            mask[:10,10:20]=1
            importants = importants.flatten(); mask = mask.flatten()

            precision, recall, f1, support = precision_recall_fscore_support(
                mask, importants,beta=1.0, average="binary",pos_label=1)
            tn,fp,fn,tp= confusion_matrix(mask, importants).ravel()
            
            macro_PR_AUC = average_precision_score(mask,orig_current_expl.flatten(),average="macro")
            micro_PR_AUC = average_precision_score(mask,orig_current_expl.flatten(),average="weighted")

            macro_ROC_AUC = roc_auc_score(mask,orig_current_expl.flatten(),average="macro")
            micro_ROC_AUC = roc_auc_score(mask, orig_current_expl.flatten() ,average="weighted")
            
            metrics["macro"]["pr"]+=precision;
            metrics["macro"]["rec"]+=recall; 
            metrics["macro"]["f1"]+=f1; 
            metrics["micro"]["tp"]+=tp;
            metrics["micro"]["fp"]+=fp; 
            metrics["micro"]["fn"]+=fn; 
            
        #average
        micro_precision = metrics["micro"]["tp"] / (metrics["micro"]["tp"] +metrics["micro"]["fp"])
        micro_recall = metrics["micro"]["tp"] / (metrics["micro"]["tp"] +metrics["micro"]["fn"])
        micro_f1 = 2*(micro_precision*micro_recall) / (micro_precision + micro_recall)
        
        macro_precision = metrics["macro"]["pr"]/test_length 
        macro_recall =  metrics["macro"]["rec"]/test_length 
        # TODO how to compute macro f1?
        macro_f1 = 2*(macro_precision*macro_recall) / (macro_precision + macro_recall)       
        
        # TODO check macro f1
        print(ns[i] ,"\t",method, '\n\t macro: precision is {:.4f};\t  recall is {:.4f}; \t f1 is {:.4f}'.format(
            macro_precision,macro_recall,macro_f1),
            '\n\t micro: precision is {:.4f};\t  recall is {:.4f}; \t f1 is {:.4f}'.format(micro_precision,micro_recall,
            micro_f1),
              '\n\t macro AUC is {:.4f};\t micro AUC is {:.4f}'.format(macro_PR_AUC,micro_PR_AUC),
              '\n\t macro ROC is {:.4f};\t  micro ROC {:.4f};'.format(macro_ROC_AUC,micro_ROC_AUC),
              "\n\n")
    print("\n\n\n")

PseudoPeriodic_Positional_False_explenations.npy 	 SHAP_ch 
	 macro: precision is 0.1471;	  recall is 0.5000; 	 f1 is 0.2273 
	 micro: precision is 0.0500;	  recall is 0.5000; 	 f1 is 0.0909 
	 macro AUC is 0.9909;	 micro AUC is 0.9909 
	 macro ROC is 0.9995;	  micro ROC 0.9995; 


PseudoPeriodic_Positional_False_explenations.npy 	 SHAP_co 
	 macro: precision is 0.2083;	  recall is 0.5000; 	 f1 is 0.2941 
	 micro: precision is 0.0500;	  recall is 0.5000; 	 f1 is 0.0909 
	 macro AUC is 0.9497;	 micro AUC is 0.9497 
	 macro ROC is 0.9974;	  micro ROC 0.9974; 


PseudoPeriodic_Positional_False_explenations.npy 	 dCAM 
	 macro: precision is 0.4943;	  recall is 0.2590; 	 f1 is 0.3399 
	 micro: precision is 0.4839;	  recall is 0.2616; 	 f1 is 0.3396 
	 macro AUC is 0.5290;	 micro AUC is 0.5290 
	 macro ROC is 0.9685;	  micro ROC 0.9685; 






GaussianProcess_Positional_False_explenations.npy 	 SHAP_ch 
	 macro: precision is 0.3044;	  recall is 0.4460; 	 f1 is 0.3619 
	 micro: precision is 0

In [46]:
print(np_array,"\n", np_array.shape)
np_array_concat_np = np_array.reshape(1,-1)
print("\n\nafter concat",np_array_concat_np.shape)

print( "\n\nto original dim",np_array_concat_np.reshape(2,100), np_array_concat_np.reshape(2,100).shape)

[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
   54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
   72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
   90  91  92  93  94  95  96  97  98  99]
 [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
  118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
  136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
  154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
  172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
  190 191 192 193 194 195 196 197 198 199]] 
 (2, 100)


after concat (1, 200)


to original dim [[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28 

In [9]:


def visualize_single_explanation(x, w, savefig=False):
    """Visualize one time series with explanation by a heatmap
    Args:
        idx: Index of the example to produce heatmap (0-indexed)
        X_series: the X_series that needs to visualize (2d array)
        explanation: coressponding explanation weights for the X_series
        ds: the name of the dataset to explain (for annotation purpose only)

    Return: a plot of heatmap explanation for an example index in a given dataset
    """
    def transform(X):
        ma,mi = np.max(X), np.min(X)
        X = (X - mi)/(ma-mi)
        return X*100
    weight = abs(w)
    weight = transform(weight)
    # z = np.histogram(weight)
    # plt.hist(weight, bins = [0,20,40,60,80,100]) 
    # plt.title("histogram") 
    # plt.show()
    ts = np.squeeze(x)
        
    max_length1, max_length2 = len(ts),10000 #
    x1 = np.linspace(0,max_length1,num = max_length1)
    x2 = np.linspace(0,max_length1,num = max_length2)
    y1 = ts
    
    f = interp1d(x1, y1) # interpolate time series
    fcas = interp1d(x1, weight) # interpolate weight color
    weight = fcas(x2) # convert vector of original weight vector to new weight vector

    plt.scatter(x2,f(x2), c = weight, cmap = 'jet', marker='.', s= 1,vmin=0,vmax = 100)
    # plt.xlabel('Explanation for index %d, dataset %s' %(idx, ds))
    cbar = plt.colorbar(orientation = 'vertical')
    
    if savefig:
        plt.savefig('temp.pdf',format='pdf',dpi=300)
    else: plt.show()

In [ ]:
mp_exp = "../../Trang/first_experiment/explanations/rocket_shap_results/MP_Concatenated4_10.npy"


In [37]:
def visualize_single_explanation(x, w, savefig=False):
    """Visualize one time series with explanation by a heatmap
    Args:
        idx: Index of the example to produce heatmap (0-indexed)
        X_series: the X_series that needs to visualize (2d array)
        explanation: coressponding explanation weights for the X_series
        ds: the name of the dataset to explain (for annotation purpose only)

    Return: a plot of heatmap explanation for an example index in a given dataset
    """
    def transform(X):
        ma,mi = np.max(X), np.min(X)
        X = (X - mi)/(ma-mi)
        return X*100
    weight = abs(w)
    weight = transform(weight)
    # z = np.histogram(weight)
    # plt.hist(weight, bins = [0,20,40,60,80,100]) 
    # plt.title("histogram") 
    # plt.show()
    ts = np.squeeze(x)
        
    max_length1, max_length2 = len(ts),10000 #
    x1 = np.linspace(0,max_length1,num = max_length1)
    x2 = np.linspace(0,max_length1,num = max_length2)
    y1 = ts
    
    f = interp1d(x1, y1) # interpolate time series
    fcas = interp1d(x1, weight) # interpolate weight color
    weight = fcas(x2) # convert vector of original weight vector to new weight vector

    plt.scatter(x2,f(x2), c = weight, cmap = 'jet', marker='.', s= 1,vmin=0,vmax = 100)
    # plt.xlabel('Explanation for index %d, dataset %s' %(idx, ds))
    cbar = plt.colorbar(orientation = 'vertical')
    
    if savefig:
        plt.savefig('temp.pdf',format='pdf',dpi=300)
    else: plt.show()


In [24]:
CMJ_slices = [  0 , 59, 119, 178, 238, 298 ,357, 417, 476, 536, 596]
MP_slices = [  0,  16,  32,  48,  64,  80,  96, 112, 128, 144, 161]

In [92]:
# load dataset metadata
import os
w = [np.load("rocket_shap_results/CMJ_ChannelByChannel.npy", allow_pickle=True).item(),
        np.load( "dCAM_results/CMJ_explenations.npy", allow_pickle=True) 
       ]

In [93]:
tmp = (w[0]["model_outputs"] == w[0]["ground_truth_labels"])
for i  in range(3):
    acc = np.sum(tmp[i]) / w[0]["model_outputs"].shape[1]
    print("channel", i ,"has accuracy",acc)

channel 0 has accuracy 0.7877094972067039
channel 1 has accuracy 0.8491620111731844
channel 2 has accuracy 0.8100558659217877


In [171]:
from numpy.linalg import norm
dCAM = w[1]
print( dCAM[0].keys() )
ch_zero = 0; ch_one = 0; ch_two = 0
no_diffs = 0
for i in range(179):
    ts = minMax_normalization( dCAM[i]["dcam_ol"])
    if norm( ts[0] - ts[1]) < 1e-13 and norm( ts[2] - ts[1]) < 1e-13 and norm( ts[0] - ts[2]) < 1e-13:
        no_diffs +=1
    importants = np.where (ts >= 50)
    ch_zero += np.sum(importants[0]==0) 
    ch_one += np.sum(importants[0]==1)
    ch_two += np.sum(importants[0]==2)
    
print(ch_zero, ch_one, ch_two, "\t", ch_zero/ch_one, ch_one/ch_one, ch_two/ch_one, "\t\t",no_diffs)


dict_keys(['dcam_tl', 'permutation_success_tl', 'dcam_ol', 'permutation_success_ol', 'ground_truth_label', 'output_label'])
1415 1592 1420 	 0.8888190954773869 1.0 0.8919597989949749 		 56


In [110]:
for i in range(w[0]["exps"].shape[0]):
    gt = w[0]["ground_truth_labels"][i]
    ot = w[0]["model_outputs"][i]
    print( w[0]["exps"][i].shape ,gt,ot)

(3, 3, 10) ['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'] ['2' '2' '2' '2' '0' '2' '0' '2' '2' '0' '0' '0' '0' '0' '1' '2' '0' '0'
 '1' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '2' '0' '1' '1' '1' '1' '1

IndexError: index 3 is out of bounds for axis 0 with size 3